In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
!ls /data2/yt8m/code/yt8m

data_converter.py  probs.npy			rnn3_finetune_val_0919_5319.npy
loader.py	   probs_rnn3_tr_0919_1943.npy	settings.py
models.py	   __pycache__			train.py
notebooks	   radam.py			utils.py
probs_100.npy	   rnn3_0918.npy		work_dirs


In [3]:
p = np.load('/data2/yt8m/code/yt8m/sigmoid/prob_3035.npy', allow_pickle=True)

In [ ]:
p1 = np.load('/data2/yt8m/code/yt8m/rnn3_finetune_val_0919_5319.npy', allow_pickle=True)
p1[:2]

In [5]:
p2 = np.load('/data2/yt8m/code/yt8m/probs.npy', allow_pickle=True)
p2[:2]

array([[6.9659978e-01, 1.3740591e-04, 1.1694086e-06, ..., 8.6689353e-05,
        6.0337617e-05, 2.5136296e-07],
       [8.1797791e-01, 4.7672194e-07, 1.8004628e-05, ..., 1.0984526e-08,
        5.2207708e-04, 1.4610782e-07]], dtype=float32)

In [6]:
p3 = np.load('/data2/yt8m/code/yt8m/rnn3_0918.npy', allow_pickle=True)
p3[:2]

array([[8.8766348e-01, 7.4459531e-05, 1.4046773e-07, ..., 1.0157839e-05,
        3.3870427e-04, 8.4951431e-08],
       [9.9903846e-01, 4.3215941e-11, 9.2645269e-09, ..., 5.1636928e-12,
        5.8856360e-05, 4.3446281e-11]], dtype=float32)

In [7]:
p = np.average([p1, p2, p3], 0, weights=[0.7, 0.15, 0.15])

In [8]:
type(p)

numpy.ndarray

In [4]:
p[:2]

array([[9.3798575e-05, 5.0189145e-05, 8.0518454e-05, ..., 1.9511305e-04,
        2.4855012e-04, 3.9240513e-05],
       [2.0981761e-06, 4.0159794e-04, 2.7664746e-06, ..., 3.6077709e-06,
        8.2285404e-03, 3.8336297e-05]], dtype=float32)

In [5]:
df_test = pd.read_csv('/data2/yt8m/3/frame/test_ids.csv')
df_test.head()

,vid,start_time,seg_id
0,MeVj,0,MeVj:0
1,MeVj,5,MeVj:5
2,MeVj,10,MeVj:10
3,MeVj,15,MeVj:15
4,MeVj,20,MeVj:20


In [6]:
def get_classes():
    df = pd.read_csv('/data2/yt8m/3/frame/classes_1000.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

classes, _ = get_classes()

In [7]:
classes[:2]

[3, 7]

In [13]:
print(df_test.shape, p.shape)

(2038114, 3) (2038114, 1001)


In [8]:
res = []
for i in tqdm(range(1000)):
    idx = np.argsort(p[:,i])[::-1][:100000]
    best_segs = df_test.seg_id.values[idx]
    pred_str = ' '.join(best_segs)
    res.append({
        'Class': classes[i],
        'Segments': pred_str
    })


100%|██████████| 1000/1000 [05:32<00:00,  3.06it/s]


In [9]:
df_sub = pd.DataFrame(res)
df_sub.head()

,Class,Segments
0,3,Rbop:145 Rbop:95 Rbop:60 Rbop:150 Rbop:40 1shy...
1,7,HmvV:75 WEoW:35 dq0N:160 WEoW:80 WEoW:105 JNH5...
2,8,I6Z7:215 sYqm:135 sYqm:60 sYqm:125 9UGW:30 9UG...
3,11,25zW:0 RBoT:120 HQI1:105 RBoT:135 oVyD:200 rRo...
4,12,FXxf:70 tn1C:135 FXxf:95 TRsL:140 0leM:190 FXx...


In [10]:
df_sub.to_csv('sigmoid_1.csv', index=False)

In [11]:
!kaggle competitions submit -c youtube8m-2019 -f sigmoid_1.csv -m 'submit'

100%|████████████████████████████████████████| 809M/809M [03:09<00:00, 4.48MB/s]
Successfully submitted to The 3rd YouTube-8M Video Understanding Challenge

In [12]:
!ls -lh

total 5.7G
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 6.0M Sep 19 20:13 create_meta.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  41K Sep 18 15:56 eda.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 812M Sep 19 06:41 ens_0919_1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 809M Sep 19 20:12 sigmoid_1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 810M Sep 17 19:27 sub1_rnn3_local057_100000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  82M Sep 17 19:16 sub1_rnn3_local057_10000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  17M Sep 17 19:06 sub1_rnn3_local057_2000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 8.2M Sep 17 03:34 sub1_rnn3_local0.57.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 810M Sep 18 04:53 sub_ens1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  13K Sep 19 06:46 submit.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  12K Sep 19 20:13 submit-sigmoid.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 812M Sep 18 

In [19]:
del res

In [20]:
del p